In [1]:
import tabpy
import pickle
from tabpy.tabpy_tools.client import Client
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

pd.options.display.max_columns=None
client = Client('http://localhost:9004/')

**Create Calculate Field in Tableau<br>**
SCRIPT_REAL("return tabpy.query('price_prediction',<br>
_arg1,_arg2,_arg3,_arg4,_arg5,_arg6,_arg7,_arg8,<br>
_arg9,_arg10,_arg11,_arg12,_arg13)['response']",<br>
[flat type input],[storey range input],<br>
[house size input],[nearest mall input],[nearest school input],<br>
[nearest mrt input],[nearest park input],[month remaining input],<br>
[mrt flag input ],[time to cbd flat input],[matured estate flag input],<br>
[town input],[flat model input]<br>
)

In [4]:
xgb_model=pickle.load(open("project_model.pickle.dat", "rb"))

In [5]:
xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=750, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [8]:
df = pd.read_csv('Final_dataset_cleaned.csv',index_col=0)
df['year'] = pd.to_datetime(df['month']).dt.year.astype(int)
df['month'] = pd.to_datetime(df['month']).dt.month.astype(int)

In [9]:
one_hot_encoded = pd.get_dummies(df[['town', 'flat_model']])
df = pd.concat([df, one_hot_encoded], axis = 1)
X = df[['month', 'year', 'flat_type_coded', 'storey_range_coded', 'floor_area_sqm', 'Dist to Nearest Mall', \
         'Dist to Nearest Pri Sch', 'Dist to Nearest Mrt/Lrt', 'Dist to Nearest Park', 'months_remaining',\
       'MRT?', 'time_to_cbd_from_flat', 'Matured Estate?', 'CPI', 'SORA', 'GDP_Q-1'] + [col for col in one_hot_encoded.columns]]
Y = df['resale_price']
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 123, test_size = 0.2)
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)
X_scaled_combined = pd.concat([pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index), pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)])
y_combined = pd.concat([y_train, y_test])


In [10]:
y_pred=xgb_model.predict(X_scaled_combined)
print('R-Squared score of model (combined):', r2_score(y_pred, y_combined))
print("Mean squared error (combined):", mean_squared_error(y_pred, y_combined))

R-Squared score of model (combined): 0.9921939701429695
Mean squared error (combined): 215588436.88594902


In [11]:
def price_prediction(_arg1,_arg2,_arg3,_arg4,_arg5,_arg6,_arg7,_arg8,_arg9,_arg10,_arg11,_arg12,_arg13):
    import numpy as np
    import pandas as pd
    import requests
    import datetime
   #create the dataframe
    df=pd.DataFrame({'flat_type':_arg1,'storey_range':_arg2,'floor_area_sqm':_arg3,
               'Dist to Nearest Mall':_arg4, 'Dist to Nearest Pri Sch':_arg5, 'Dist to Nearest Mrt/Lrt':_arg6,
               'Dist to Nearest Park':_arg7,'months_remaining':_arg8,'MRT?':_arg9,'time_to_cbd_from_flat':_arg10,
               'Matured Estate?':_arg11})

    town=_arg12[0]
    flat_model=_arg13[0]

   #data transformation
    flat_type_dict={'4 ROOM': 4,'5 ROOM': 5,'3 ROOM': 3,'EXECUTIVE': 7,'2 ROOM': 2,'MULTI-GENERATION': 6,'1 ROOM': 1}
    storey_range_dict={'04 TO 06': 2,'07 TO 09': 3,'10 TO 12': 4,'01 TO 03': 1,'13 TO 15': 5,'16 TO 18': 6,'19 TO 21': 7,
   '22 TO 24': 8,'25 TO 27': 9,'28 TO 30': 10,'31 TO 33': 11,'34 TO 36': 12,'37 TO 39': 13,'40 TO 42': 14,'43 TO 45': 15,
   '46 TO 48': 16,'49 TO 51': 17}
    df['flat_type_coded']=df["flat_type"].map(flat_type_dict)
    df['storey_range_coded']=df["storey_range"].map(storey_range_dict)

   #get the latest Singapore Overnight Rate Average indicator
    url = "https://eservices.mas.gov.sg/api/action/datastore/search.json?resource_id=9a0bf149-308c-4bd2-832d-76c8e6cb47ed&limit=5&fields=end_of_day,sora&sort=end_of_day desc"
    response = requests.get(url)
    data = response.json()
    records = pd.DataFrame(data["result"]["records"])
    sora=records.loc[records["sora"].first_valid_index(), "sora"]
    df['SORA']=sora

   #get the latest CPI & GDP 
    url1 = 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M212881?sortby=key%20desc&limit=1'
    cpiresponse = requests.get(url1)
    cpi_data = cpiresponse.json()
    df['CPI']= cpi_data['Data']['row'][0]['columns'][0]['value'] 

    url2 = 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M014811?sortby=key%20desc&limit=1'
    gdpresponse = requests.get(url2)
    gdp_data = gdpresponse.json()
    df['GDP_Q-1']= gdp_data['Data']['row'][0]['columns'][0]['value']
   
   #get the latest year & month
    df['year'] = datetime.date.today().year
    df['month'] = datetime.date.today().month

   #one-hot encoding
    town_encoding=['town_ANG MO KIO', 'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK',
       'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH',
       'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI',
       'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST',
       'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS',
       'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
       'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
       'town_YISHUN']
    df_town=pd.DataFrame(dict(zip(town_encoding, len(town_encoding)*[0])),index=[0])
    col=[i for i in df_town.columns if town in i]
    df_town[[i for i in df_town.columns if town in i][0]]=1

    flat_encoding=['flat_model_2-room', 'flat_model_3Gen',
       'flat_model_Adjoined flat', 'flat_model_Apartment', 'flat_model_DBSS',
       'flat_model_Improved', 'flat_model_Improved-Maisonette',
       'flat_model_Maisonette', 'flat_model_Model A',
       'flat_model_Model A-Maisonette', 'flat_model_Model A2',
       'flat_model_Multi Generation', 'flat_model_New Generation',
       'flat_model_Premium Apartment', 'flat_model_Premium Apartment Loft',
       'flat_model_Premium Maisonette', 'flat_model_Simplified',
       'flat_model_Standard', 'flat_model_Terrace', 'flat_model_Type S1',
       'flat_model_Type S2']
    df_flat=pd.DataFrame(dict(zip(flat_encoding, len(flat_encoding)*[0])),index=[0])
    col=[i for i in df_flat.columns if flat_model in i]
    df_flat[[i for i in df_flat.columns if flat_model in i][0]]=1

    df2=pd.concat([df_town,df_flat],axis=1)

   #rearrange
    df = df[['month', 'year', 'flat_type_coded', 'storey_range_coded', 'floor_area_sqm', 'Dist to Nearest Mall', \
   'Dist to Nearest Pri Sch', 'Dist to Nearest Mrt/Lrt', 'Dist to Nearest Park', 'months_remaining',\
   'MRT?', 'time_to_cbd_from_flat', 'Matured Estate?', 'CPI', 'SORA', 'GDP_Q-1'] ]
    df_final=pd.concat([df,df2],axis=1)

    X=sc.transform(df_final)
    X=pd.DataFrame(X, columns=df_final.columns)

    y_pred=xgb_model.predict(X)

    return y_pred.tolist()

In [12]:
client.deploy('price_prediction',price_prediction,"This function is used to forecast resale HDB flat price",override = True)